<a href="https://colab.research.google.com/github/aaronmat1905/the-tokenized-tech/blob/main/LLM%20Engineering/Agentic%20AI/AgentsByHand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**The Tokenized Tech**\
**Article 1:** Write an AI Agent Like It's 1999

In [1]:
# !pip install -q dotenv

In [30]:
# Code for Google-colab platforms
from google.colab import userdata
api_key = userdata.get("GeminiAPI") or "YOUR-GEMINI-API-KEY"

**Gemini Client** \
*Custom Implementation*

In [62]:
# Custom Gemini Client:
import os
import requests

# from dotenv import load_env
# api_key = os.getenviron("GEMINI_API_KEY")

class GeminiClient:
	def __init__(self, api_key: str):
		self.api_key = api_key
		self.endpoint = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent"
		self.headers = {
			"x-goog-api-key": self.api_key,
			"Content-Type": "application/json"
			}
	def generate_content(self, prompt) -> str:
		payload = {
			"contents": [
					{ "parts":[{"text":prompt}]}
				]
		}
		response = requests.post(self.endpoint, headers= self.headers, json = payload)
		if response.status_code ==200:
			data = response.json()
			try:
				return data['candidates'][0]['content']['parts'][0]['text']
			except (KeyError, IndexError):
				return "No response text found"
		else:
			return f"Error: {response.status_code} - {response.text}"

In [55]:
# Useage of our custom client:
client = GeminiClient(api_key)

In [63]:
# Sample Prompt:
prompt = "Explain what are AI Agents in a sentence. in Malayalam"
response = client.generate_content(prompt)
print(f"Response: {response}")

Response: ലക്ഷ്യങ്ങൾ നേടാനായി സ്വയം ചിന്തിച്ച് പ്രവർത്തിക്കുന്ന ആർട്ടിഫിഷ്യൽ ഇൻ്റലിജൻസ് സംവിധാനങ്ങളാണ് AI ഏജൻ്റുമാർ.


**Defining Our Tools**

In [34]:
# Tool Registry:
class Tool:
  def __init__(self):
    self.tools = {}
  def registerTool(self, name=None, description=None):
    "Records a Tool"
    def decorator(func):
      tool_name = name or func.__name__
      self.tools[tool_name] = {
          "function_name":func,
          "description":description or func.__doc__,
          "call": "tool_name(*args, **kwargs)"
      }
      return func
    return decorator

  def call(self, tool_name:str, *args,**kwargs):
    if tool_name not in self.tools:
      raise ValueError(f"Tool '{tool_name}' does not exist.")
    return self.tools[tool_name]["function_name"](*args, **kwargs)
  def list_tools(self):
    return {name:meta["description"] for name, meta in self.tools.items()}

In [35]:
# Creating the Tool Registry:
toolFamily = Tool()

**Problem Statement:**  Create an Agent for Personal Financial Wellness & Life Planning

**Emergency Fund Calculator** \
Figures out how much money you should save for emergencies.

In [36]:
@toolFamily.registerTool()
def calculate_emergency_fund_goal(monthly_expenses, job_stability, dependents):
    """
    Calculate personalized emergency fund target based on life situation.
    Args:
        monthly_expenses (float): Monthly living expenses.
        job_stability (str): "stable", "moderate", "unstable" - job security level.
        dependents (int): Number of dependents (spouse, kids, parents).
    Returns:
        dict: Emergency fund recommendations and savings plan.
    """
    base_months = 3

    # Adjust based on job stability
    if job_stability == "stable":
        months_needed = base_months
    elif job_stability == "moderate":
        months_needed = base_months + 2
    else:  # unstable
        months_needed = base_months + 4

    # Add buffer for dependents
    months_needed += dependents * 0.5

    target_amount = monthly_expenses * months_needed
    monthly_savings_needed = target_amount / 12  # Build over 1 year

    return {
        "target_amount": round(target_amount, 2),
        "months_covered": round(months_needed, 1),
        "monthly_savings_goal": round(monthly_savings_needed, 2),
        "weekly_savings_goal": round(monthly_savings_needed / 4, 2),
        "motivation": f"This covers {months_needed} months of expenses - you'll sleep better!"
    }

**Vacation Savings Planner**\
Purpose: Helps you save money for your next trip with weekly goals.

In [37]:
@toolFamily.registerTool()
def calculate_vacation_savings_plan(destination, trip_cost, departure_date, current_savings):
    """
    Create a personalized vacation savings plan with weekly goals.
    Args:
        destination (str): Where you're going (for motivation).
        trip_cost (float): Total estimated trip cost.
        departure_date (str): When you're leaving (YYYY-MM-DD).
        current_savings (float): Amount already saved.
    Returns:
        dict: Savings plan with weekly targets and motivation.
    """
    from datetime import datetime

    departure = datetime.strptime(departure_date, "%Y-%m-%d")
    today = datetime.now()
    days_until_trip = (departure - today).days
    weeks_until_trip = days_until_trip / 7

    amount_still_needed = trip_cost - current_savings

    if weeks_until_trip <= 0:
        return {"error": "Trip date has passed or is today!"}

    weekly_savings_needed = amount_still_needed / weeks_until_trip

    progress_percentage = (current_savings / trip_cost) * 100

    return {
        "destination": destination,
        "days_until_trip": days_until_trip,
        "weekly_savings_needed": round(weekly_savings_needed, 2),
        "daily_savings_needed": round(weekly_savings_needed / 7, 2),
        "progress_percentage": round(progress_percentage, 2),
        "motivation": f"Just ${round(weekly_savings_needed, 2)} per week until you're in {destination}!",
        "coffee_equivalent": f"That's about {round(weekly_savings_needed / 5, 0)} coffees per week"
    }

**Buy vs Rent Analyzer**\
Purpose: Shows whether it's better to buy or rent a home for your situation.

In [38]:
@toolFamily.registerTool()
def calculate_buying_vs_renting(home_price, rent_price, down_payment, years_staying, annual_home_appreciation):
    """
    Compare the financial impact of buying vs renting for your situation.
    Args:
        home_price (float): Purchase price of the home.
        rent_price (float): Monthly rent cost.
        down_payment (float): Down payment amount.
        years_staying (int): How long you plan to stay.
        annual_home_appreciation (float): Expected annual home value growth percentage.
    Returns:
        dict: Detailed buy vs rent analysis with recommendation.
    """
    # Buying costs
    loan_amount = home_price - down_payment
    monthly_mortgage = loan_amount * 0.005  # Simplified 6% rate
    property_tax = home_price * 0.015 / 12  # 1.5% annually
    insurance = home_price * 0.005 / 12  # 0.5% annually
    maintenance = home_price * 0.01 / 12  # 1% annually

    monthly_buying_cost = monthly_mortgage + property_tax + insurance + maintenance

    # Future home value
    future_home_value = home_price * ((1 + annual_home_appreciation/100) ** years_staying)

    # Total costs over period
    total_buying_cost = (monthly_buying_cost * years_staying * 12) + down_payment
    total_renting_cost = rent_price * years_staying * 12

    # Equity gained
    equity_gained = future_home_value - loan_amount
    net_buying_cost = total_buying_cost - equity_gained

    savings = total_renting_cost - net_buying_cost

    recommendation = "Buy" if savings > 0 else "Rent"

    return {
        "monthly_mortgage_payment": round(monthly_mortgage, 2),
        "total_monthly_ownership_cost": round(monthly_buying_cost, 2),
        "monthly_rent": rent_price,
        "equity_after_years": round(equity_gained, 2),
        "total_savings_by_buying": round(savings, 2),
        "recommendation": recommendation,
        "break_even_years": round(abs(savings) / (abs(monthly_buying_cost - rent_price) * 12), 1),
        "personal_advice": f"If you're staying {years_staying} years, {recommendation.lower()}ing saves you ${abs(round(savings, 2))}"
    }

**Coffee Shop Savings Calculator**\
Purpose: Shows how much you could save by making coffee at home instead of buying it.

In [39]:
@toolFamily.registerTool()
def calculate_coffee_shop_savings(daily_coffee_cost, brewing_cost_per_cup, workdays_per_week):
    """
    Calculate how much you could save by making coffee at home vs buying.
    Args:
        daily_coffee_cost (float): Cost of your daily coffee shop visit.
        brewing_cost_per_cup (float): Cost to make coffee at home.
        workdays_per_week (int): How many days per week you buy coffee.
    Returns:
        dict: Savings analysis with fun comparisons.
    """
    daily_savings = daily_coffee_cost - brewing_cost_per_cup
    weekly_savings = daily_savings * workdays_per_week
    monthly_savings = weekly_savings * 4.33  # Average weeks per month
    annual_savings = monthly_savings * 12

    # Fun comparisons
    netflix_subscriptions = annual_savings / 180  # ~$15/month
    vacation_fund = annual_savings / 2000  # Decent vacation cost

    return {
        "daily_savings": round(daily_savings, 2),
        "weekly_savings": round(weekly_savings, 2),
        "monthly_savings": round(monthly_savings, 2),
        "annual_savings": round(annual_savings, 2),
        "netflix_equivalent": round(netflix_subscriptions, 1),
        "vacation_fund_portion": round(vacation_fund, 2),
        "fun_fact": f"In 10 years, you'd save ${round(annual_savings * 10, 2)} - that's a car!",
        "motivation": f"Skip the coffee shop {workdays_per_week} days a week = ${round(monthly_savings, 2)} extra per month"
    }

**Subscription Audit Tool**\
Purpose: Finds subscriptions you're paying for but not really using.

In [40]:
@toolFamily.registerTool()
def calculate_subscription_audit(subscriptions_list, monthly_usage_hours):
    """
    Analyze your subscription spending and find savings opportunities.
    Args:
        subscriptions_list (list): List of dicts with 'name', 'cost', 'frequency' (monthly/annual).
        monthly_usage_hours (dict): Dict with subscription names as keys, usage hours as values.
    Returns:
        dict: Subscription analysis with recommendations.
    """
    total_monthly_cost = 0
    underused_subs = []

    for sub in subscriptions_list:
        monthly_cost = sub['cost'] if sub['frequency'] == 'monthly' else sub['cost'] / 12
        total_monthly_cost += monthly_cost

        usage = monthly_usage_hours.get(sub['name'], 0)
        cost_per_hour = monthly_cost / usage if usage > 0 else monthly_cost

        if usage < 5:  # Less than 5 hours per month
            underused_subs.append({
                'name': sub['name'],
                'monthly_cost': round(monthly_cost, 2),
                'usage_hours': usage,
                'cost_per_hour': round(cost_per_hour, 2)
            })

    potential_savings = sum([sub['monthly_cost'] for sub in underused_subs])

    return {
        "total_monthly_cost": round(total_monthly_cost, 2),
        "annual_cost": round(total_monthly_cost * 12, 2),
        "underused_subscriptions": underused_subs,
        "potential_monthly_savings": round(potential_savings, 2),
        "potential_annual_savings": round(potential_savings * 12, 2),
        "recommendation": f"Cancel {len(underused_subs)} underused subscriptions to save ${round(potential_savings, 2)}/month",
        "reality_check": f"You're spending ${round(total_monthly_cost, 2)}/month on subscriptions - that's a car payment!"
    }

In [41]:
## Let us now see how these tools are registered in our system: Creating a repository of our tools:
toolFamily.list_tools()

{'calculate_emergency_fund_goal': '\n    Calculate personalized emergency fund target based on life situation.\n    Args:\n        monthly_expenses (float): Monthly living expenses.\n        job_stability (str): "stable", "moderate", "unstable" - job security level.\n        dependents (int): Number of dependents (spouse, kids, parents).\n    Returns:\n        dict: Emergency fund recommendations and savings plan.\n    ',
 'calculate_vacation_savings_plan': "\n    Create a personalized vacation savings plan with weekly goals.\n    Args:\n        destination (str): Where you're going (for motivation).\n        trip_cost (float): Total estimated trip cost.\n        departure_date (str): When you're leaving (YYYY-MM-DD).\n        current_savings (float): Amount already saved.\n    Returns:\n        dict: Savings plan with weekly targets and motivation.\n    ",
 'calculate_buying_vs_renting': '\n    Compare the financial impact of buying vs renting for your situation.\n    Args:\n        h

In [42]:
## Let us call each of the tool we have created:
toolFamily.call('calculate_emergency_fund_goal', 3500, 'moderate', 2)

{'target_amount': 21000.0,
 'months_covered': 6.0,
 'monthly_savings_goal': 1750.0,
 'weekly_savings_goal': 437.5,
 'motivation': "This covers 6.0 months of expenses - you'll sleep better!"}

In [43]:
# Vacation Savings Planner:
toolFamily.call('calculate_vacation_savings_plan', 'Hawaii', 5000, '2025-12-15', 1200)

{'destination': 'Hawaii',
 'days_until_trip': 150,
 'weekly_savings_needed': 177.33,
 'daily_savings_needed': 25.33,
 'progress_percentage': 24.0,
 'motivation': "Just $177.33 per week until you're in Hawaii!",
 'coffee_equivalent': "That's about 35.0 coffees per week"}

In [44]:
toolFamily.call('calculate_buying_vs_renting', 400000, 2500, 80000, 5, 3.5)

{'monthly_mortgage_payment': 1600.0,
 'total_monthly_ownership_cost': 2600.0,
 'monthly_rent': 2500,
 'equity_after_years': 155074.52,
 'total_savings_by_buying': 69074.52,
 'recommendation': 'Buy',
 'break_even_years': 57.6,
 'personal_advice': "If you're staying 5 years, buying saves you $69074.52"}

In [45]:
toolFamily.call('calculate_coffee_shop_savings', 5.50, 0.75, 5)

{'daily_savings': 4.75,
 'weekly_savings': 23.75,
 'monthly_savings': 102.84,
 'annual_savings': 1234.05,
 'netflix_equivalent': 6.9,
 'vacation_fund_portion': 0.62,
 'fun_fact': "In 10 years, you'd save $12340.5 - that's a car!",
 'motivation': 'Skip the coffee shop 5 days a week = $102.84 extra per month'}

In [46]:
toolFamily.call('calculate_subscription_audit',
    [
        {'name': 'Netflix', 'cost': 15.99, 'frequency': 'monthly'},
        {'name': 'Spotify', 'cost': 9.99, 'frequency': 'monthly'},
        {'name': 'Adobe Creative', 'cost': 239.88, 'frequency': 'annual'}
    ],
    {'Netflix': 20, 'Spotify': 40, 'Adobe Creative': 2}
)

{'total_monthly_cost': 45.97,
 'annual_cost': 551.64,
 'underused_subscriptions': [{'name': 'Adobe Creative',
   'monthly_cost': 19.99,
   'usage_hours': 2,
   'cost_per_hour': 9.99}],
 'potential_monthly_savings': 19.99,
 'potential_annual_savings': 239.88,
 'recommendation': 'Cancel 1 underused subscriptions to save $19.99/month',
 'reality_check': "You're spending $45.97/month on subscriptions - that's a car payment!"}

**Thought-Action-Observation** Cycle

**Approach**:\
Every Agent works in a Thought-Action-Observation Cycle.

**Thought**: The Agent first reads the Query and plans the perfect set of steps that needs to be executed in order to get the query's output.

Here, We try to adopt a prompt-engineering approach, where we specify the list of tools that would be used for our project. The LLM would read our query and list the most appropriate tool call/s to get the output.

In [47]:
# Thought Prompt:
def buildThoughtPrompt(work, toollist:Tool) -> str:
  agentPrompt = f"You are an AI-system equipped with some tools. You are an expert {work}"
  outputGeneration = """
    You are expected to plan and reason the steps that need to be taken in order to arrive at the expected answer.
    Each step of your thinking should be shown as a dictionary element contained within a Python list.
    Each step should only contain the work related to the tool and nothing else.
    If the query can be resolved within a single step- go for it; Else document each step in the format given below.
    ```python
    [
      {
        "thought": "a-one-sentence-description-of-why-you-selected-this-tool",
        "action": (Tool_Name, *args, **kwargs)
      },
      ...
    ]
    ```
    Only generate this python object- do not generate any extra code or explainatory text
    Here are your Tools.
  """
  agentPrompt += outputGeneration
  for key, val in toolFamily.list_tools().items():
    samp = f"\nTool Name:\t{key}\nTool Description:\n{val}\n"
    agentPrompt += samp
  return agentPrompt

**Action**: Are the concrete steps an AI Agent Takes to interact with it's environment

In [80]:
# First, we would want our Agent to be a seperate object to make things easier:
class Agent():
  def __init__(self, work, toolfamily, llm_client):
    self.work = work
    self.toolfamily = toolfamily
    self.thought_prompt = buildThoughtPrompt(work, toolfamily)
    self.llm = llm_client

  def _think(self, query) -> list:
    """Runs the Thinking Step of the Agent"""
    prompt = f"{self.thought_prompt}\n\nHere is your Query {query}"
    answer = self.llm.generate_content(prompt)
    answer = answer.replace("python", "").replace("```", "")
    return eval(answer)

  def _act(self, thought: list[dict]) -> list:
      " Runs the Action step of the Agent"
      results = []
      for step in thought:
          action_tuple = step.get("action")
          if not action_tuple:
              continue  # Skip malformed steps
          tool_name, *args = action_tuple
          result = self.toolfamily.call(tool_name)(*args)
          results.append(result)
      return results
  def run(self, query):
    thoughts = self._think(query)
    data = self._act(thoughts)
    return data

In [81]:
financeAgent = Agent(
    work = "Financial Agent",
    toolfamily= toolFamily,
    llm_client= client
)

In [85]:
sampleQuestion = """
  I wanna take a little vacation to Sri Lanka where the trip cost is approx INR 45,000; I would like to leave on the second week of december-- I have INR 3000 in my a/c.. Would I be able to take the trip?
"""
# dct = financeAgent.run(sampleQuestion)